In [1]:
%matplotlib widget

# MicroCT Image Processing

In [2]:
#
# Copyright 2020- IBM Inc. All rights reserved
# SPDX-License-Identifier: BSD-3-Clause
#

## About this Notebook

This companion notebook is published along a data-centric journal publication [1] related to a published Open-Source dataset on cabonate and sandstone rock tomographies [2] and aims to provide readers and users of the dataset with a way to visualize the data, as well as to the algorithms used for processing of the microCT image data.

This notebook also demonstrates the difference between using the Otsu and multi-Otsu algorithms for determining the porosity of the samples. Further information about these algorithms and their effect on the porosity estimates can be found in [1].

### References 

[1] Add reference to paper

[2] 10.25452/figshare.plus.21375565

## Importing Libraries

In [3]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
from skimage.filters import threshold_multiotsu, threshold_otsu

### Digital Rock Class

The digital rock class contains the algorithm used for contrast filtering of the rock samples. An instance to this class takes as input the path of the raw NxNxN cube file and the number of pixels on each size of the cube, N.

It is important to note, however, that this code won't work as is for the full paralellpided files, as it expects cubic volumes. This can be easily circonvented, but may be unusable due to memory requirements.

The equalize function carries out the contrast enhancement filter as described on [1]. As default, the filter cuts off the histogram at the 99.8 percentile, but this can be changed in the "clip" variable.

In [4]:
class digitalRock():
    def __init__(self, path, N):
        self.N = N
        self.volume = np.fromfile(path, dtype=np.uint8).reshape([N,N,N])
        
    def multp(self, A):
        return np.uint8((A*self.scale)*255)
    
    def equalize(self, clip=99.8):
        self.filter_thr = int(np.percentile(self.volume, clip))
        clipped_volume = np.clip(self.volume, 0, self.filter_thr)
        self.minInt = clipped_volume.min()
        self.maxInt = clipped_volume.max()
        self.scale = 1/(self.maxInt-self.minInt)
        
        clipped_volume -= self.minInt
        filtered = np.array(list(map(self.multp,clipped_volume)))
        
        return filtered

### Visualizations

In [5]:
def plot_slices(img, N=2500, scale=True):
    @widgets.interact(Zslice=(0,N-1,1))
    def update(Zslice = 0):
        plt.figure()
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('Z={}'.format(Zslice))
        if scale==True:
            plt.imshow(img[:,:,Zslice], vmin=0, vmax=255)
        else:
            plt.imshow(img[:,:,Zslice])
        plt.colorbar()
        plt.show()

In [6]:
def plot_hist(img, N=2500):
    @widgets.interact(Zslice=(0,N-1,1))
    def update(Zslice = 0):
        plt.figure()
        plt.title('Z={}'.format(Zslice))
        plt.hist(img[:,:,Zslice].ravel(), bins=255, range=[0,255])
        plt.show()

 ## Loading Data                                              

In [7]:
filename = 'kocurek_5a_2p25um_ir_rec_2500x2500x2500_grayscale_filtered_ROI-1.raw'
N=2500

In [15]:
rock = digitalRock(filename, N)

### Selecting an Image Subset

Due to the large memory requirements for loading the full microCT images in this work, the next cell can be used to select a subset of the original raw volume (given by N). In case you prefer to visualize a subset of the image, commenting the next cell will be enough. It is important to note that the memory requirements can be up to 8x the image size (due to conversions between uint8 and float 64 by numpy).

In [ ]:
# N = 1000
# rock.volume = rock.volume[:N, :N,:N]

## Data Visualization

In [21]:
plot_slices(rock.volume, N=N)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

In [22]:
plot_hist(rock.volume, N=N)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

## Contrast Enhancement Filter

In [23]:
filtered = rock.equalize()

### Before

In [24]:
plot_slices(rock.volume, N=N)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

In [25]:
plot_hist(rock.volume, N=N)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

### After

In [26]:
plot_slices(filtered, N=N)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

In [27]:
plot_hist(filtered, N=N)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

## Anisotropic Diffusion

As discussed in the methods section [1], the images generated from the last step were later processed by an anisotropic diffusion filter implemented into the CT analyser software (Bruker, version 1.20.8.0). The filter was set to 3D space, the type used was Privilege high contrast edges (Perona-Malik), the number of iterations set to 5 and the gradient threshold set to 10. The user defined integration constant was left unchecked. This step reduces the amount of image noise that is inherent in any digital imaging technique.

## Image Segmentation

### Otsu

In [28]:
thr_otsu = threshold_otsu(filtered)

In [29]:
binary_otsu = filtered >= thr_otsu

In [30]:
plot_slices(binary_otsu, N=N, scale=False)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

### Multilevel Otsu

In [31]:
thr_multiotsu = threshold_multiotsu(filtered)

In [32]:
regions = np.digitize(filtered, bins=thr_multiotsu)

In [33]:
plot_slices(regions, N=N, scale=False)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

In [34]:
thr_multiotsu

array([107, 142])

In [45]:
binary_multiotsu = filtered >= thr_multiotsu[0]

In [36]:
plot_slices(binary_multiotsu, N=N, scale=False)

interactive(children=(IntSlider(value=0, description='Zslice', max=2499), Output()), _dom_classes=('widget-int…

### Calculating Porosity

In [37]:
porosity_otsu = 1-np.mean(binary_otsu)
print('Calculated Porosity for Otsu = {} %'.format(np.round(porosity_otsu, 2)*100))

Calculated Porosity for Otsu = 32.0 %


In [38]:
porosity_multiotsu = 1-np.mean(binary_multiotsu)
print('Calculated Porosity for MultiOtsu = {} %'.format(np.round(porosity_multiotsu, 2)*100))

Calculated Porosity for MultiOtsu = 9.0 %


## Data Export

If you wish to save the filtered data, just execute the code below for saving the new image to the folder

In [ ]:
# filtered.tofile(workdir + filename.replace('grayscale', 'binary'))
